In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import os
import shutil
from IPython.display import FileLink
import yaml
import wandb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
FileLink('/kaggle/working/yolov5')

/kaggle/working/yolov5

In [3]:
# %pip install comet_ml

In [4]:
# shutil.rmtree('/kaggle/working/yolo')

In [5]:
os.makedirs('/kaggle/working/dataset/yolo_type')

In [6]:
import os
import xml.etree.ElementTree as ET

def convert_annotation(xml_path, image_width, image_height):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    class_list = ['without_mask', 'with_mask', 'mask_weared_incorrect']
    yolo_annotations = []
    
    for obj in root.findall('object'):
        class_name =class_list.index(obj.find('name').text)
#         print(class_name)
        bbox = obj.find('bndbox')
        xmin = float(bbox.find('xmin').text)
        ymin = float(bbox.find('ymin').text)
        xmax = float(bbox.find('xmax').text)
        ymax = float(bbox.find('ymax').text)
        
        # Calculate bounding box coordinates and normalize them
        x_center = (xmin + xmax) / (2.0 * image_width)
        y_center = (ymin + ymax) / (2.0 * image_height)
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height
        
        yolo_annotations.append(f"{class_name} {x_center} {y_center} {width} {height}")
#     print(yolo_annotations)
    return yolo_annotations

def convert_dataset(xml_dir, image_dir, output_dir):
#     os.makedirs(output_dir, exist_ok=True)

    xml_files = os.listdir(xml_dir)
    for xml_file in xml_files:
        xml_path = os.path.join(xml_dir, xml_file)
        image_file = xml_file.replace('.xml', '.png')
        image_path = os.path.join(image_dir, image_file)
        
        if not os.path.exists(image_path):
            continue

        image = Image.open(image_path)
        image_width, image_height = image.size
        
        yolo_annotations = convert_annotation(xml_path, image_width, image_height)

        # Save the YOLO annotations to a text file
        txt_filename = xml_file.replace('.xml', '.txt')
        txt_path = os.path.join(output_dir, txt_filename)
        
        with open(txt_path, 'w') as f:
            
            f.write('\n'.join(yolo_annotations))


In [7]:
from PIL import Image

# Sample data paths
xml_dir = '/kaggle/input/face-mask-detection/annotations'  # Directory containing XML annotation files
image_dir = '/kaggle/input/face-mask-detection/images'  # Directory containing corresponding images
output_dir = '/kaggle/working/dataset/yolo_type'  # Directory to save the YOLO annotations

# Call the convert_dataset function
convert_dataset(xml_dir, image_dir, output_dir)


In [8]:
li = os.listdir('/kaggle/working/dataset/yolo_type')

In [9]:
def get_number_from_filename(file_name):
    return int(file_name.split('.')[0].split('s')[-1])
# Sort the files based on the extracted number
sorted_files = sorted(li, key=get_number_from_filename)

In [10]:
train_directory = os.path.join('/kaggle/working/dataset', 'train/labels')
os.makedirs(train_directory, exist_ok=True)
test_directory = os.path.join('/kaggle/working/dataset', 'test/labels')
os.makedirs(test_directory, exist_ok=True)
train_directory_img = os.path.join('/kaggle/working/dataset', 'train/images')
test_directory_img = os.path.join('/kaggle/working/dataset', 'test/images')

os.makedirs(train_directory_img, exist_ok=True)
os.makedirs(test_directory_img, exist_ok=True)


In [11]:
train_files, test_files = train_test_split(sorted_files, test_size=0.1, random_state=42)
print(len(train_files),len(test_files))
data_directory = '/kaggle/working/dataset/yolo_type'
train_directory = '/kaggle/working/dataset/train/labels'
test_directory ='/kaggle/working/dataset/test/labels'
train_img_directory = '/kaggle/working/dataset/train/images'
test_img_directory ='/kaggle/working/dataset/test/images'

def store_label_to_folder(directory,files):
    for file_name in files:
        current_path = os.path.join(data_directory, file_name)
        new_path = os.path.join(directory, file_name)
        shutil.copy(current_path, new_path)
        
store_label_to_folder(train_directory,train_files)
store_label_to_folder(test_directory,test_files)

767 86


In [12]:
def add_images_to_folders(label_path,target_dir):
    c=0
    for file in list(os.listdir(label_path)):
        c+=1
#         print(file)
        new_file_name = file.replace('.txt', '.png') 
        current_path = os.path.join(image_dir, new_file_name)
        new_path = os.path.join(target_dir, new_file_name)
        shutil.copy(current_path, new_path)
#     print(c)

add_images_to_folders('/kaggle/working/dataset/train/labels',train_img_directory)
add_images_to_folders('/kaggle/working/dataset/test/labels',test_img_directory)

In [13]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 15996, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 15996 (delta 101), reused 111 (delta 69), pack-reused 15831
Receiving objects: 100% (15996/15996), 14.64 MiB | 15.96 MiB/s, done.
Resolving deltas: 100% (10974/10974), done.


In [14]:
%cd yolov5

/kaggle/working/yolov5


In [15]:
os.listdir()

['classify',
 'train.py',
 'val.py',
 '.pre-commit-config.yaml',
 '.gitignore',
 'tutorial.ipynb',
 'setup.cfg',
 '.gitattributes',
 'detect.py',
 'hubconf.py',
 'benchmarks.py',
 'CITATION.cff',
 'data',
 'requirements.txt',
 'utils',
 'models',
 '.github',
 'CONTRIBUTING.md',
 'README.zh-CN.md',
 '.dockerignore',
 'README.md',
 'LICENSE',
 'segment',
 '.git',
 'export.py']

In [16]:
# FileLink('/kaggle/working/dataset/test/images/maksssksksss338.png')

In [17]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.0/804.0 kB 43.8 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 59.8.0
    Uninstalling setuptools-59.8.0:
      Successfully uninstalled setuptools-59.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-api 1.17.0 requires importlib-metadata~=6.0.0, but you have importlib-metadata 5.2.0 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.23.5 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.10.1 which is incompatible.


In [18]:
%cd ..

/kaggle/working


In [19]:
pwd()

'/kaggle/working'

In [20]:
!source yolo/bin/activate

/bin/bash: line 1: yolo/bin/activate: No such file or directory


In [21]:
%cd yolov5

/kaggle/working/yolov5


In [22]:
!pip install -r requirements.txt

In [23]:
!/kaggle/working/yolov5/weights/download_weights.sh

/bin/bash: line 1: /kaggle/working/yolov5/weights/download_weights.sh: No such file or directory


In [24]:
pwd()

'/kaggle/working/yolov5'

In [25]:
os.listdir('/kaggle/working/yolov5')

['classify',
 'train.py',
 'val.py',
 '.pre-commit-config.yaml',
 '.gitignore',
 'tutorial.ipynb',
 'setup.cfg',
 '.gitattributes',
 'detect.py',
 'hubconf.py',
 'benchmarks.py',
 'CITATION.cff',
 'data',
 'requirements.txt',
 'utils',
 'models',
 '.github',
 'CONTRIBUTING.md',
 'README.zh-CN.md',
 '.dockerignore',
 'README.md',
 'LICENSE',
 'segment',
 '.git',
 'export.py']

In [26]:
%cd /kaggle/working/yolov5

/kaggle/working/yolov5


In [27]:
!ls

CITATION.cff	 README.zh-CN.md  detect.py   requirements.txt	tutorial.ipynb
CONTRIBUTING.md  benchmarks.py	  export.py   segment		utils
LICENSE		 classify	  hubconf.py  setup.cfg		val.py
README.md	 data		  models      train.py


In [28]:
%cd data

/kaggle/working/yolov5/data


In [29]:
%cd scripts/

/kaggle/working/yolov5/data/scripts


In [30]:
!ls

download_weights.sh  get_coco.sh  get_coco128.sh  get_imagenet.sh


In [31]:
!/kaggle/working/yolov5/data/scripts/download_weights.sh

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ModuleNotFoundError: No module named 'utils'


In [32]:
# !cat download_weights.sh

In [33]:
%cd data

[Errno 2] No such file or directory: 'data'
/kaggle/working/yolov5/data/scripts


In [34]:
# !cat coco128.yaml

In [35]:
with open('coco128.yaml', 'r') as file:
    yaml_data = yaml.safe_load(file)

# Now you have the YAML data as a dictionary
print(yaml_data)


FileNotFoundError: [Errno 2] No such file or directory: 'coco128.yaml'

In [ ]:
yaml_data['path'] = '/kaggle/input/face-mask-detection'
yaml_data['train'] = '/kaggle/working/dataset/train/images'
yaml_data['val'] = '/kaggle/working/dataset/test/images'
del yaml_data['test']


In [ ]:
yaml_data['names']={
    0:'without_mask',
    1:'with_mask',
    2:'mask_weared_incorrect'
}

In [ ]:
yaml_data

In [ ]:
%cd ../datasets/coco128

In [ ]:
file=open("/kaggle/working/yolov5/coco128.yaml","w")
yaml.dump(yaml_data,file)
file.close()


from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

experiment = Experiment(
  api_key = "iLnNG9eEIRN38vbmJaQB4Xefs",
  project_name = "maskdetection2",
  workspace="saireddy"
)
# Report multiple hyperparameters using a dictionary:
hyper_params = {
   "learning_rate": 0.5,
   "steps": 100000,
   "batch_size": 50,
}
experiment.log_parameters(hyper_params)

# Initialize and train your model
# model = TheModelClass()
# train(model)

# Seamlessly log your Pytorch model




!python train.py --img 416 --batch 16 --epoch 600 --data coco128.yaml --weights yolov5l.pt --nosave --cache
log_model(experiment, model=None, model_name="TheModel")

In [ ]:
!python train.py --img 416 --batch 16 --epoch 100 --data coco128.yaml --weights yolov5l.pt --nosave --cache

In [ ]:
%cd ..

In [ ]:
%cd train

In [ ]:
cd exp2/

In [ ]:
%cd weights

In [ ]:
!ls

In [ ]:
%cd ..

In [ ]:
os.listdir('/kaggle/working/yolov5/runs/train/exp2')

In [ ]:
!python val.py --weights /kaggle/working/yolov5/runs/train/exp2/weights/last.pt --data coco128.yaml --img 416

In [ ]:
!ls /kaggle/working/yolov5/runs/val/exp3

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image

#read the image
# im = Image.open('/kaggle/working/yolov5/runs/val/exp3/val_batch0_labels.jpg')
import matplotlib.image as mpimg
img = mpimg.imread('/kaggle/working/yolov5/runs/detect/exp/labels')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
!cat /kaggle/working/yolov5/runs/detect/exp/labels/face-mask-study.txt

In [ ]:
!wandb input

In [ ]:
!wandb login

In [ ]:
!ls

In [ ]:
%cd ..

## !python train.py --img 416 --batch 16 --epoch 100 --data coco128.yaml --weights yolov5l.pt --nosave --cache
input_value = 2

# Monkey-patch the input() function to return the desired value
wandb.util._console_input = lambda prompt=None: input_value

In [ ]:
!python detect.py --source /kaggle/input/Newimgs1  --weights /kaggle/working/yolov5/runs/train/exp2/weights/last.pt --img 416 --save-txt --save-conf

In [ ]:
!ls runs/detect/exp4/labels

In [ ]:
import cv2

In [ ]:
img = cv2.imread('/kaggle/input/testimg/face-mask-study.jpg')

In [ ]:
# 0.7125 0.301389 0.134375 0.286111 0.958417

start_point = (round(71), round(30))
print(start_point)

# 1 0.7125 0.301389 0.134375 0.286111 0.958417
# 0 0.325391 0.305556 0.132031 0.288889 0.966178
# 1 0.711719 0.8 0.123438 0.263889 0.968723
# 0 0.299609 0.798611 0.125781 0.255556 0.980087


# Ending coordinate, here (220, 220)
# represents the bottom right corner of rectangle
end_point = (round(13.4375) ,round(28.6111))
  
# Blue color in BGR
color = (255, 0, 0)
  
# Line thickness of 2 px
thickness = 2
  
# Using cv2.rectangle() method
# Draw a rectangle with blue line borders of thickness of 2 px
cv2.rectangle(img, start_point, end_point, color, thickness)
cv2.imwrite('/kaggle/working/1.png',img)

In [ ]:
# Read the annotation file (assuming it's in YOLO format with class index, normalized box coordinates)
# with open('/kaggle/working/yolov5/runs/detect/exp/labels/face-mask-study.txt', 'r') as file:
# Define a dictionary mapping class indices to shape names
class_mapping = {
    0: 'circle',
    1: 'rectangle',
    2: 'triangle',
    # Add more class index to shape name mappings as needed
}

# Read the annotation file (assuming it's in YOLO format with class index, normalized box coordinates)
with open('/kaggle/working/yolov5/runs/detect/exp/labels/face-mask-study.txt', 'r') as file:
    lines = file.readlines()

# Extract the bounding box coordinates and associated shape name
for line in lines:
    # Split the line by spaces
    data = line.strip().split(' ')
    
    # Extract the class index and normalized box coordinates
    class_index = int(data[0])
    x_center = float(data[1])
    y_center = float(data[2])
    width = float(data[3])
    height = float(data[4])
    
    # Convert the normalized box coordinates to actual pixel values
    image_width = 416  # Width of the input image (adjust this according to your setup)
    image_height = 416  # Height of the input image (adjust this according to your setup)
    
    box_width = width * image_width
    box_height = height * image_height
    
    # Calculate the top-left corner coordinates of the bounding box
    top_left_x = (x_center * image_width) - (box_width / 2)
    top_left_y = (y_center * image_height) - (box_height / 2)
    
    # Retrieve the shape name based on the class index
    shape_name = class_mapping.get(class_index)
    
    # Print the shape name and bounding box shape
    print(f"Shape name: {shape_name}")
    print(f"Bounding box shape: {box_width} x {box_height}")


In [ ]:
# import cv2

# # Read the image
image=img
# image = cv2.imread('your_image.jpg')  # Replace 'your_image.jpg' with the actual image file path

# Iterate through each annotation line
with open('/kaggle/working/yolov5/runs/detect/exp/labels/face-mask-study.txt', 'r') as file:
    lines = file.readlines()

for line in lines:
    # Split the line by spaces
    data = line.strip().split(' ')
    
    # Extract the class index and bounding box coordinates
    class_index = int(data[0])
    x_center = float(data[1])
    y_center = float(data[2])
    width = float(data[3])
    height = float(data[4])
    
    # Convert the normalized coordinates to pixel values
    image_height, image_width, _ = image.shape
    
    box_width = int(width * image_width)
    box_height = int(height * image_height)
    
    top_left_x = int((x_center * image_width) - (box_width / 2))
    top_left_y = int((y_center * image_height) - (box_height / 2))
    
    # Define the bounding box coordinates
    bottom_right_x = top_left_x + box_width
    bottom_right_y = top_left_y + box_height
    
    # Draw the bounding box on the image
    color = (0, 255, 0)  # Green color (you can change it as desired)
    thickness = 2  # Line thickness (you can change it as desired)
    cv2.rectangle(image, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), color, thickness)
    
    # Add class label to the bounding box
    class_list = ['without_mask', 'with_mask', 'mask_weared_incorrect']
    cls = class_list[class_index]
    label = f"Class {cls}"
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.8
    text_color = (0, 255, 0)  # Green color (you can change it as desired)
    text_thickness = 2
    cv2.putText(image, label, (top_left_x, top_left_y - 10), font, font_scale, text_color, text_thickness)
    
# Display the annotated image
cv2.imwrite('/kaggle/working/1.png',image)


